<pre>
<h1>
Binary Search Tree, BST
</h1>
[root, size, height] --------> Node()

보통 이진트리에서 값을 찾으려면 순회하며 모든 노드를 돌아다녀야 했다. 뽈뽈뽈
이때 세가지 방법이 있었다. preorder, inorder, postorder

하지만, 애초에 규칙을 정해두고 이진트리를 만든다면? 찾기도 쉬울것이다.
따라서, 탐색 연산의 효율성을 위해 일정 규칙에 맞게 값을 삽입할 수 있다.

<h1>
1. Node Class
</h1>
-구조
    [parent, key, (height), left, right]

특수 메서드
__init__ 
__iter__
__str__


<h1>
2. BST Class
</h1>
-구조
    [root, size, height]

- 규칙
    1) None값은 하나의 텅 빈 BST로 취급한다.
    2) BST의 노드를 v라고 했을때, v는 왼쪽 자식노드(left)보다 커야하고, 오른쪽 자식노드(right)보다는 작아야한다. 
       left < v < right

- 메서드 명세
    특수 메서드
    __init__        => root, size, (height) 
    __len__
    __iter__        => Node Class에 정의한 이터레이터 호출
    __str__         


    일반 메서드
    preorder(v)
    inorder(v)
    postorder(v)
    
    findLocation(key)   => hashTable을 구현할 때, 어떤 key값이 들어갈 자리, 혹은 이미 들어있는 자리를 찾았던 것처럼(findSlot)
                        => (들어간 자리) 규칙에 따라 내려갔을 떄 일치하는 key값이 존재하면 해당 노드를 반환하고, 
                        => (들어갈 자리) 비어있으면 삽입될 곳의 부모를 리턴
    search(key)         => v와 v의 자손노드 중에서 주어진 key 값과 일치하는 노드를 찾고(findLocation) 없다면 None리턴
    insert(key)         => key가 들어갈 자리가 정해져있으므로, findLocation을 호출
                        => key값이 찾은 노드의 키와 일치하지 않고, None이 아니라면 내가 들어갈 부모노드
                            => 비교해서 왼쪽이나 오른쪽에 삽입
    updateNodeHeight(v) => 노드 v의 현재 높이 수정
    updateHeight(v)     => v부터 root까지 모든 노드의 높이 수정 => updateNodeHeight(v) 호출
    deleteByMerging(x)
    deleteByCopying(x)
</pre>

In [227]:
class Node:
    def __init__(self, key):
        self.key = key
        self.height = 1
        self.parent = None
        self.left = None
        self.right = None
    
    # preorder 방식으로 순회
    def __iter__(self):
        if self != None:
            yield self
            if self.left != None:
                for leftSubTree in self.left:
                    yield leftSubTree
            if self.right != None: 
                for rightSubTree in self.right:
                    yield rightSubTree
                
    def __str__(self):
        return str(self.key)

class BST:
    def __init__(self):
        self.root = None
        self.size = 0
        self.height = 0
    def __len__(self):
        return self.size
    
    def __str__(self):
        if not self.root:
            return "empty"
        # 레벨별로 노드들을 저장할 리스트 생성
        result = []
        current_level = [self.root]  # 루트에서부터 시작

        while current_level:
            # 각 레벨의 노드를 문자열로 변환
            level_str = "Lv" + str(self.height - current_level[0].height+1) + ": " + \
                        ", ".join(str(node) if node else "None" for node in current_level)
            result.append(level_str)

            # 다음 레벨 노드들로 이동
            next_level = []
            for node in current_level:
                if node:
                    next_level.append(node.left)
                    next_level.append(node.right)

            # 다음 레벨로 이동 (모두 None이면 종료)
            if any(next_level):
                current_level = next_level
            else:
                break
        return "\n".join(result)
        #if self.root:
        #    return ("\n").join("Lv"+str(self.height-i.height+1)+" : "+str(i) for i in self.root)
        #else:
        #    return "empty"
        
    def preorder(self, v):
        if v != None:
            print(v, end=" ")
            if v.left != None:
                self.preorder(v.left)
            if v.right != None:
                self.preorder(v.right)

    def inorder(self, v):
        if v != None:
            if v.left != None:
                self.inorder(v.left)
            print(v, end=" ")
            if v.right != None:
                self.inorder(v.right)
                
    def postorder(self, v):
        if v != None:
            if v.left != None:
                self.postorder(v.left)
            if v.right != None:
                self.postorder(v.right)
            print(v, end=" ")
                
    def findLocation(self, key):
        # 들어있는 자리를 찾거나, 내가 들어갈 자리를 확인 후, 부모를 호출(그래야 부모의 자식으로 연결할 수 있으니까)
        # 근데 애초에 빈 트리면 부모도 내자리도 아무것도 없으니 None
        if self.size == 0: return None
        # running tech을 사용하여, 어떤 노드 v와 v의 부모노드인 p를 동시에 찾는다.
        p = None 
        v = self.root
        # v가 None이 되면 
        while v:
            # key값이 일치하는 v를 찾았다면 반환
            if v.key == key:
                return v
            # 찾고자 하는 key가 v의 key값보다 크다면 오른쪽 탐색
            elif v.key < key:
                p = v
                v = v.right
            # 찾고자 하는 key가 v보다 작으면 왼쪽 탐색
            else:
                p = v
                v = v.left
        # while문을 빠져나왔다는 것은, 찾고자 하는 key를 가진 v가 존재하지 않고, 해당 노드가 들어갈 자리만 찾았다는 말
        # 따라서 v가 들어갈 수 있는 자리의 부모노드 반환
        return p
    
    # 사실 findLocation의 일정부분까지만 사용하면 search를 구현할 수 있지만, 코드의 재사용성을 위해 
    # 왜냐면 해당 노드를 찾는다고 해도, key값이 들어갈 자리의 부모를 찾기 위해 똑같은 걸 반복해야하니까.
    def search(self, key):
        #findLocation을 호출하여, 해당 key가 들어가있거나, 들어갈 자리의 부모노드를 호출(만약 아무것도 없으면 None)
        p = self.findLocation(key)
        if p.key == key:
            return p
        else: 
            return None
    # 노드 v의 높이를 수정
    def update_node_height(self, v):
        # v가 존재한다면,
        if v:
            # 해당 노드의 자식이 존재하는지 검사 후, 자식 중에 더 깊이 뻗어있는 자식을 선택해서 현재 높이를 업데이트
            l = v.left.height if v.left else -1
            r = v.right.height if v.right else -1
            # 최대 높이를 가진 자식보단 본인이 한칸 더 크니까
            v.height = max(l, r) +1
            if self.height < v.height:
                self.height = v.height
    # v부터 root가지 올라가면서 모든 노드 정보 업데이트
    # 노드를 삽입할 때마다 호출한다면, 언제나 리프노드부터 루트까지 존재하는 높이들을 업데이트 가능
    def update_height(self, v):
        # v가 루트노드에 도달할때까지 
        while v != None:
            # 높이를 갱신하고, 부모노드로 현재노드 이동
            self.update_node_height(v)
            v = v.parent
    
    def insert(self, key):
        p = self.findLocation(key)
        # 삽입이 가능한 경우 : 빈 노드에 첫 삽입이거나, 부모 노드가 반환된 경우
        if p == None or p.key != key:
            # print("검색 성공")
            newNode = Node(key)
            self.size += 1
            # 첫 삽입인 경우
            if p == None:
                # print("첫번째 삽입")
                self.root = newNode
                self.root.height =1
                self.height =1
            # 내가 들어갈 부모 노드를 찾은 경우
            else:
                # print("하나 이상의 노드가 존재하는 트리에 삽입 시도")
                newNode.parent = p
                # 부모보다 작은 값은 왼쪽에 삽입
                if p.key > key:
                    # print("왼쪽 자식에 연결")
                    p.left = newNode
                # 부모보다 큰 경우 오른쪽
                else:
                    # print("오른쪽 자식에 연결")
                    p.right = newNode
                self.update_height(newNode)
        # 삽입하고자 하는 key값이 이미 존재하는 경우
        else:
            # print("이미 중복된 값이 존재")
            # 중복 허용 안한다면 none
            return None

    def 

    def printInfo(self):
        print("\n=>  info")
        tree_structure = str(self).split("\n")  # __str__에서 나온 결과를 줄 단위로 나눔
        for line in tree_structure:
            print("\n    - Tree Structure  | " + line, end="")  # 각 줄에 들여쓰기 추가
        print("\n    - Postorder       | ", end="")
        self.postorder(self.root)
        print("\n    - Tree height     | ",  self.height)
        print("    - Number of Node  | ", self.size)

In [228]:
B = BST()
print(B)
B.insert(4)
B.insert(43)
B.insert(5)
B.insert(2)
B.insert(3)
B.insert(1)
B.insert(51)

B.printInfo()

B.postorder(B.root)

empty

=>  info

    - Tree Structure  | Lv1: 4
    - Tree Structure  | Lv2: 2, 43
    - Tree Structure  | Lv3: 1, 3, 5, 51
    - Postorder       | 1 3 2 5 51 43 4 
    - Tree height     |  2
    - Number of Node  |  7
1 3 2 5 51 43 4 